In [ ]:
from bs4 import *
import requests as rq
import os

In [ ]:
'''
r2 = rq.get("https://www.airbnb.com.tw/")
soup2 = BeautifulSoup(r2.text,"html.parser")

links = []

x = soup2.find_all("div")
print(x)
for tag in x:
  y = tag.find_all("img")
  for img in y:
    print(img['src'])


print("end")
'''
!mkdir /content/sample_data/images
!mkdir /content/sample_data/process_img

In [ ]:
import signal
from contextlib import contextmanager
import os
import time
import json
import requests

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

def downloadFromURL(imagesURL):
    i = 0
    for img in imagesURL:  
      i += 1
      try:
        with time_limit(10):
          img_data = requests.get(img[1]).content
          print("saving image {}".format(i))
          with open("/content/sample_data/images/{}.jpg".format(img[0]), 'wb') as handler:
            handler.write(img_data)
      except TimeoutException as e:
        print("Timed out!")      

def getManyPages(pages):
    params=[]
    for i in range(0, 12*pages+12, 12):
        params.append({
            'resource_id': 28266,
            'from_mid': 1,
            'format': 'json',
            'ie': 'utf-8',
            'oe': 'utf-8',
            'query': '明星',
            'sort_key': '',
            'sort_type': 1,
            'stat0': '',
            'stat1': '',
            'stat2': '',
            'stat3': '',
            'pn': i,
            'rn': 12
                  })
    url = 'https://sp0.baidu.com/8aQDcjqpAAV3otqbppnN2DJv/api.php'
    names = []
    img_results = []
    x = 0
    f = open('starName.txt', 'w')
    for param in params:
        try:
            res = requests.get(url, params=param)
            js = json.loads(res.text)
            results = js.get('data')[0].get('result')
        except:
            continue
        for result in results:
            img_name = result['ename']
            img_url = result['pic_4n_78']
            img_result =  [img_name,img_url]
            img_results.append(img_result)
            f.write(img_name+'\n')
            #print(img_name+"\n")
            names.append(img_name)

        if x % 10 == 0:
            print('第%d页......'%x)
        x += 1
    f.close()
    #downloadFromURL(img_results)
    #for img in img_results:
      #print("0: ",img[0])
      #print("1: ",img[1])
    downloadFromURL(img_results)  

getManyPages(1000)



In [ ]:
import cv2
import os


def processImage(path,new_path):  
  face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
  for file in os.listdir(path):
    print(file)  
    if file.endswith(".jpg") or file.endswith(".png"):
      try:
        print(file) 
        img = cv2.imread("{}/{}".format(path,file))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        for (x, y, w, h) in faces:
          #cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
          faces = img[y:y + h, x:x + w]
          faces = cv2.resize(faces, (64, 64), interpolation=cv2.INTER_AREA)
          cv2.imwrite("{}/{}".format(new_path,file), faces)
      except:
        continue    
      cv2.waitKey()  

processImage(path='/content/sample_data/images',new_path='/content/sample_data/process_img')

In [ ]:
!pip install facenet-pytorch

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import torch
def detectFaceImage(save_count = 5000):
  count = 0
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
  )
  resnet = InceptionResnetV1(pretrained='vggface2').eval()
  path = "/content/sample_data/process_img"
  for file in os.listdir(path):
    image_dir = path+"/"+file
    x = Image.open(image_dir)
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is None:
      print(file)
      os.remove(image_dir)
    else:
      print("prob :",prob)
      count+=1
      if(count >= save_count):
          break       

detectFaceImage(6000)

In [ ]:
!apt-get install rar
!rar a "/content/sample_data/celebA_asia.rar" "/content/sample_data/process_img"